<a href="https://colab.research.google.com/github/xDaryamo/BreastCancerPrediction/blob/master/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import json
import os
import spacy
from spacy import displacy
from tabulate import tabulate
from string import punctuation
import re

In [36]:
#!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_trf --quiet
!pip install spacy-transformers --quiet

2022-11-27 15:20:05.171339: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     |████████████████████████████████| 460.3 MB 21 kB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 1.1 MB 10.8 MB/s 
     |████████████████████████████████| 4.7 MB 57.6 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 182 kB 59.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [45]:
import spacy_transformers
nlp = spacy.load("en_core_web_trf")

In [46]:
def features(sentence):

    
    doc = nlp(sentence)
    tokens = []
    tokens_dep = []
    tokens_pos = []

    for token in doc:
      if str(token) not in punctuation:
        if token.ent_type_ == '':
          tokens.append(token.text)
        else:
          tokens.append(token.ent_type_)

    for token in doc:
      if str(token) not in punctuation:
        tokens_dep.append(token.dep_)
 
    for token in doc:
        if str(token) not in punctuation:
            tokens_pos.append(token.pos_)
 
    return tokens, tokens_dep, tokens_pos 

In [47]:
def is_quote_ok(s):
    stack = []
    for c in s:
        if c in ["'", '"', "`"]:
            if stack and stack[-1] == c:
                stack.pop()
            else:
                stack.append(c)
        else:
            # ignore it
            pass

    return len(stack) == 0

In [48]:
def findWholeWord(keyword, string):
  return re.search(r'\b' + keyword + r'\W', string)

In [49]:
def is_member(sentence):
  if re.search("^(\(?[0-9]+\s*)[\)\-](\s+)?", sentence):
    sentence = re.sub("^(\(?[0-9]+\s*)[\)\-](\s+)?", '', sentence)
    flag = True
  elif re.search("^([\-]+)(\s+)?", sentence):
    sentence = re.sub("^([\-]+)(\s+)?", '', sentence)
    flag = True
  elif re.search("^(\(?[a-zA-Z]*)\)+(\s+)?", sentence):
    sentence = re.sub("^(\(?[a-zA-Z]*)\)+(\s+)?", '', sentence)
    flag = True
  elif re.search("^[a-z]\.+[^g](\s+)?", sentence):
    sentence = re.sub("^[a-z]\.+[^g](\s+)?", '', sentence)
    flag = True
  elif re.search("^(• )+(\s+)?", sentence):
    sentence = re.sub("^(• )+(\s+)?", '', sentence)
    flag = True
  else:
    flag = False
    
  
  return flag, sentence

In [50]:
#Titoli paragrafi e sezioni
def is_title(sentence):
  return re.search("^((I|i)(N|n)|(D|d)(C|c))?\.?([0-9])([\.0-9]+)", sentence) or re.search("^[A-Za-z]+\s+((S|s)ection)", sentence)

In [51]:
def is_skippable(sentence):

  #ID delle User Stories
  if re.search("^(User Story: )?US[0-9]+(\.*[0-9])*", sentence) or re.search("^(Technical Story: )?TS[0-9]+(\.[0-9]*)*", sentence):
    return True
    
  #Solo caratteri speciali
  elif re.search("^[\W_]+$", sentence) or re.search("^([\|\.\;\,\@][\d\s\w]?)+", sentence) or re.search("^\s*\+", sentence):
    return True

  #Tutte le frasi composte da solo 2 parole
  elif len(re.findall('\w+', sentence)) <= 2:
    return True

  #Le frasi 'None at present.' ricorrenti nel dataset
  elif sentence == "None at present.":
    return True

  #Url
  elif re.search("^https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$", sentence) or \
        (re.search("^([a-z\-\_]+\.)+[a-z]+", sentence) and not re.search("^(e\.g\.|E\.g\.)\s*", sentence)):
    return True

  #Didascalie tabelle e immagini
  elif re.search("^(Figure|Table)\s?[0-9]", sentence):
    return True

  #Testi "See also"
  elif re.search("^(See also|see also)", sentence):
    return True

  #Email
  elif re.search("^[a-zA-Z0-9.! #$%&'*+/=? ^_`{|}~-]+@[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)*$", sentence):
    return True

  #Log message
  elif re.search("^((L|l)og (m|M)essage|(f|F)eature (r|R)equest|(F|f)eature (I|i)(d|D))", sentence):
    return True

  #Informazioni temporali
  elif re.search("^(build date:)", sentence) or \
        re.search("(?:\"|\')?((J|j)an(?:uary)?|(F|f)eb(?:ruary)?|(M|m)ar(?:ch)?|(A|a)pr(?:il)?|(M|m)ay|(J|j)un(?:e)?|(J|j)ul(?:y)?|(A|a)ug(?:ust)?|(S|s)ept(?:ember)?|(S|s)ep(?:tember)?|(O|o)ct(?:ober)?|(N|n)ov(?:ember)?|(D|d)ec(?:ember)?)\s+(\d{1,2}(st|nd|rd|th)?)\,?\s+((\d{4})\s?)?((\d{1,2}):\d{1,2}\s+(am|AM|Am|pm|Pm|PM)?)?", sentence) or \
        re.search("\d{4,4}-\d{1,2}-\d{1,2}\s+((\d{1,2}):\d{1,2}\s+(am|AM|Am|pm|Pm|PM)?)?", sentence) or \
        re.search("^build date:", sentence) or re.search("^(\d(-| to | or )?\s?)+((d|D)ays?|(m|M)onths? | (y|Y)ears? | (w|W)eeks?)", sentence) or \
        re.search("^[\d]+\s(business )?(days?|weeks?|years?|months?)", sentence):
    return True
  
  #Frasi che indicano errori (iniziano con "issue x")
  elif re.search("^(Issues?)\s+\d+", sentence):
    return True

  elif re.search("^(org.|sun.|javax.|oscar.|net.|lang.|doinhibernate|qcvl.|qeh.|qgoatway.)", sentence):
    return True

  elif re.search("^(Please note)[^:]*:$", sentence) or re.search("(go to):?\s*(https?:\/\/)?(\w\w\w.)?[a-zA-z]+", sentence):
    return True

  elif re.search("^(login|log in|logon|log on|logout|log out|(c|C)urrent functionality:|(r|R)equired new functionality:|WL:|(I|i)ntake \d*)", sentence):
    return True

  #Metodi o funzioni
  elif re.search("^(insert into|include)", sentence) or re.search("^([A-Za-z]+)+\([\W\w\d]*\);?", sentence) or re.search("^(add|Add|ADD)", sentence) or re.search("^((C|c)lick on)", sentence) or re.search("^((G|g)et)", sentence) or \
        re.search("^(O?SCAR)", sentence) or re.search("^((P|p)lease)", sentence) or re.search("^((S|s)elect)", sentence) or re.search("^((M|m)ove)", sentence) or re.search("^((S|s)etup:)", sentence) or\
        re.search("^((C|c)hange)", sentence) or re.search("^((S|s)how)", sentence) or re.search("^((U|u)pdate)", sentence) or re.search("^((R|r)emove)", sentence) or re.search("^((I|i)nclude)", sentence) or\
        re.search("^[\'\"\`]?[A-Za-z\s]+[\'\"\`]?\=\s*[\'\"\`]?\_[\w\d]+[\'\"\`]?", sentence) or re.search("^[\w\d\s]+(\-\>)[\w\d\s]+[\.\;\?\!]?$", sentence) or re.search("^((F|f)ile (A|a)dd(ed)?:?)", sentence) or\
        re.search("^[A-Za-z\s]+\=[A-Za-z\s\W]+$", sentence) or re.search("^((B|b)uild (T|t)ag)\s?\:", sentence):
    return True

  elif re.search("^\d(st|nd|rd):?", sentence):
    return True
  else:
    return False

In [52]:
def clean_data(sentence, s_type, old_sentence, list_start):
  
  #Rimuoviamo i caratteri � dalle stringhe e gli spazi all'inizio e alla fine
  sentence = sentence.replace('�', '').replace("", '').replace("\‘", "\"").replace("\’", "\"").replace("\“", "\"").replace("\”", "\"")
  old_sentence = old_sentence.replace('�', '').replace("", '').replace("\‘", "\"").replace("\’", "\"").replace("\“", "\"").replace("\”", "\"")
  sentence = sentence.strip("\"").strip("\'").strip("\`").strip()
  old_sentence = old_sentence.strip("\"").strip("\'").strip("\`").strip()

  
  
  if re.search("^\([\w\W]*\)$", sentence):
    sentence = sentence.strip().strip("(").strip(")")
  elif re.search("^\([\w\W]*\)$", old_sentence):
    old_sentence = old_sentence.strip().strip("(").strip(")")

  flag_member, new_sentence = is_member(old_sentence)
  #Se è un titolo
  if (s_type == 'TITLE' or is_title(old_sentence)) and not flag_member:
    #print("SKIPPO: " + sentence)
    return None, None


  #Se è un inizio di una lista 
  elif s_type == 'LIST_START' and not re.search("^[\W_]+$", sentence):
    list_start = sentence.strip("-")
    return None, list_start
  
  


  #Se è il membro di una lista
  if (s_type == 'LIST_MEMBER' or flag_member) and not re.search("^[\W_]+$", sentence):
    if list_start == None:
      sentence = new_sentence.strip()
    else: 
      sentence = list_start + ' ' + new_sentence



  if sentence != None:
    #Se la frase non rispetta certi requisiti non viene considerata\
    if is_skippable(sentence):
      #print("SKIPPO: " + sentence)
      return None, list_start
    else:   
      sentence = re.sub('^[\"\'\`][\"\'\`][\.\;\,]?$', '', sentence)
      if(not is_quote_ok(sentence)):
        sentence = sentence.replace("\"", '').replace("\'", '').replace("\`", '')
      sentence = re.sub('[\;\.\,\:]$', '', sentence)
      sentence = re.sub('[\s][\s]+', ' ', sentence)
      sentence = re.sub('(\_|-){2,255}','', sentence)
      if sentence[-1] != '?' and sentence[-1] != '!':
        sentence = sentence + "."
      sentence = sentence[0].upper() + sentence[1:]

  
  return sentence, list_start
 

  

In [53]:
def get_keywords_list(sentence):
  listk = []
  to_remove = []
  for keyword in keywords:
      if findWholeWord(keyword, sentence.lower()):
            listk.append(keyword) 

  if not listk:
    return listk
    
  for i in range(0, len(listk)-1):
    for j in range(i+1, len(listk)):
      if listk[i] in listk[j] and listk[i] not in to_remove:
        to_remove.append(listk[i])
      elif listk[j] in listk[i] and listk[j] not in to_remove:
        to_remove.append(listk[j])

  return [x for x in listk if x not in to_remove]

In [54]:
root = 'https://raw.githubusercontent.com/xDaryamo/NFR-Security-Extraction-Classification/master/'


keywords = pd.read_excel(root + "security_words.xlsx").values.tolist()
kws = []
for keyword in keywords:
  kws.append(keyword[0])
keywords = kws

# Dataset


In [57]:
!wget https://github.com/xDaryamo/NFR-Security-Extraction-Classification/raw/master/riaz_dataset/riaz_dataset.zip -O riaz.zip
!mkdir riaz
!unzip riaz.zip -d riaz/

--2022-11-27 15:22:29--  https://github.com/xDaryamo/NFR-Security-Extraction-Classification/raw/master/riaz_dataset/riaz_dataset.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/xDaryamo/NFR-Security-Extraction-Classification/master/riaz_dataset/riaz_dataset.zip [following]
--2022-11-27 15:22:29--  https://raw.githubusercontent.com/xDaryamo/NFR-Security-Extraction-Classification/master/riaz_dataset/riaz_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 520520 (508K) [application/zip]
Saving to: ‘riaz.zip’

riaz.zip            100%[===================>] 508.32K  --.-KB/s    in 0.04s 

In [58]:
from pandas.io.common import dataclasses
dataset_path = './riaz'
df = pd.DataFrame(columns=["Sentence", "Entities", "Dependencies", "Parts of Speech", "File", "Categories", "Security Words", "Security"])

counter = 0
list_start = None
for file in os.listdir(dataset_path):
    print("Processing " + file + " ...")
    with open(dataset_path+'/'+file) as f:
      data = json.load(f)
      
    for i in range(0, len(data['content'])):
      current_item = data['content'][i]
      sentence = current_item['parserSentence']
      full_sentence = current_item['sentence']
      s_type = current_item['sentenceType']
      ojective = current_item['securityObjectiveAnnotations']
      sentence, list_start = clean_data(sentence, s_type, full_sentence,  list_start)


      if not sentence:
        continue

      #print(sentence)

      entities, dependencies, pos = features(sentence)
      df.loc[counter, "Sentence"] = sentence
      df.loc[counter,"Entities"] = entities
      df.loc[counter,"Dependencies"] = dependencies
      df.loc[counter,"Parts of Speech"] = pos
      df.loc[counter,"File"] = current_item['documentID']


      if not ojective or (len(ojective)) == 1 and (ojective[0]["securityObjective"] == 'DATABASE' or \
                                                    ojective[0]["securityObjective"] == 'TECHNICAL' or \
                                                    ojective[0]["securityObjective"] == 'MANAGEMENT'):

        df.loc[counter,"Security"] = 0
        df.loc[counter, "Categories"] = "none"

      else:
        df.loc[counter,"Security"] = 1
        categories = []
        for item in ojective:
          if item["securityObjective"] != 'DATABASE' and item["securityObjective"] != 'TECHNICAL' and item["securityObjective"] != 'MANAGEMENT':
            if item["securityObjective"] == 'AVAILABILITY_SURVIVABILITY':
              if 'AVAILABILITY' not in categories:
                categories.append('AVAILABILITY')
            elif item["securityObjective"] == 'INTEGRITY_IMMUNITY':
              if 'INTEGRITY' not in categories:
                categories.append('INTEGRITY')
            else:
              categories.append(item["securityObjective"])

        df.loc[counter, "Categories"] = categories

      security_words = get_keywords_list(sentence.lower())
      if not security_words:
        df.loc[counter, "Security Words"] = 'none'
      else:
        df.loc[counter, "Security Words"] = security_words

      counter+=1

df = df.drop_duplicates(subset=['Sentence'])
df.head()
   

    

Processing nursing_ehr_business_and_functional_elements_june__2012_PUBLISH.json ...
Processing HL7_Functional_Profile - PUBLISH.json ...
Processing EHR-Privacy-Security-Requirements_PUBLISH.json ...
Processing featureRequests - for annotation -PUBLISH.json ...
Processing VLER UserStories Combined_PUBLISH.json ...
Processing CCHIT Certified 2011 Ambulatory EHR Criteria 20110517_PUBLISH.json ...


,Sentence,Entities,Dependencies,Parts of Speech,File,Categories,Security Words,Security
0,The vision of healthier Canadians through inno...,"[The, vision, of, healthier, NORP, through, in...","[det, nsubjpass, prep, amod, pobj, prep, amod,...","[DET, NOUN, ADP, ADJ, PROPN, ADP, ADJ, NOUN, N...",nursing_ehr,"[CONFIDENTIALITY, AVAILABILITY, INTEGRITY]",[access],1
1,"Once completed, the electronic health record (...","[Once, completed, the, electronic, health, rec...","[advmod, advcl, det, amod, compound, nsubj, ap...","[ADV, VERB, DET, ADJ, NOUN, NOUN, PROPN, AUX, ...",nursing_ehr,none,none,0
2,Canada Health Infoway's (Infoway's) Nursing Re...,"[ORG, ORG, ORG, ORG, ORG, 's, ORG, ORG, ORG, O...","[compound, compound, poss, case, poss, case, c...","[PROPN, PROPN, PROPN, PART, PROPN, PART, PROPN...",nursing_ehr,none,none,0
3,Timely access to information will help inform ...,"[Timely, access, to, information, will, help, ...","[amod, nsubj, prep, pobj, aux, ROOT, xcomp, de...","[ADJ, NOUN, ADP, NOUN, AUX, VERB, VERB, DET, A...",nursing_ehr,[AVAILABILITY],[access],1
4,"As part of the NRG, a pan-Canadian working gro...","[As, part, of, the, ORG, a, NORP, NORP, workin...","[prep, pobj, prep, det, pobj, det, amod, amod,...","[ADP, NOUN, ADP, DET, PROPN, DET, ADJ, ADJ, VE...",nursing_ehr,none,none,0


# Esportazione del dataset

In [59]:
df.to_excel('/content/drive/MyDrive/Security Extraction/dataset.xlsx',index=False)

# Esempio con una frase

In [ ]:
#df.loc[df['Security'] == 'TRUE'].iloc[0,0]
nfr = "The specifics of this issue is not addressed by any of the requirements, although Privacy Requirement 8, Privacy Requirement 9, Privacy Requirement 10, Privacy Requirement 11, and Privacy Requirement 12 specify that consent data be captured by POS systems and transmitted to the EHRi when transmitting the underlying PHI. Facebook"
doc = nlp(nfr)
displacy.render(doc, style='ent', jupyter=True)
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
data = [] 
for token in doc:
    if str(token) not in punctuation:
        data.append([token.text, token.pos_, token.dep_])

print(tabulate(data, headers=["Text", "Part of speech", "Dependency"]))

In [ ]:
df = df.drop_duplicates(subset=['Sentence'])
df.head(25)

NameError: ignored